# Relax Data Science Challenge
## Springboard
### Federico Di Martino

In [1]:
## import packages
import pandas as pd
import numpy as np
import random
random.seed( 123456789 )
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression


## Wrangling
# import the data and show head
engagement = pd.read_csv('takehome_user_engagement.csv', encoding='latin-1')
engagement.head()

users = pd.read_csv('takehome_users.csv', encoding='latin-1')
users.head()

## drop unnecessary columns and NAs
users = users[['object_id','last_session_creation_time','opted_in_to_mailing_list',    'enabled_for_marketing_drip']]
users = users.dropna()
engagement = engagement.dropna()

# conver engagment time_stamp into datetime
engagement['time_stamp']=pd.to_datetime(engagement['time_stamp'])

#find which users were adopted
# loop a 7 day rolling count of user engagement for each users
adopted=[]
for u in users.object_id:   
    count = engagement[engagement['user_id']==u]
    count = count.set_index('time_stamp')
    count = count.sort_index()
    count = count.rolling(window='7d').visited.sum().max()
    if count >= 3:
        adopted.append(1)
    else:
        adopted.append(0)

users['adopted'] = adopted


## Determining key features using linear regression
#set target
X = users.loc[:, users.columns != 'adopted']
y = users.iloc[:,4]
# split into train and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state = 123456789)

lr = LinearRegression()
lr.fit(X_train, y_train)
lr_predict = lr.predict(X_test)

# get importance
importance = lr.coef_
# summarize feature importance
for i,v in enumerate(importance):
	print('Feature: %0d, Score: %.5f' % (i,v))

Feature: 0, Score: 0.00000
Feature: 1, Score: 0.00000
Feature: 2, Score: -0.00849
Feature: 3, Score: -0.00303


The factors which predict future user adoption, 2 and 3 in the code, are opted_in_to_mailing_list and enabled_for_marketing_drip.

It is clear that marketing is the decisive factor driving user adoption.